In [1]:
import numpy as np

# ステップ11:可変長の入力に対応する

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray): # 形式がndarrayでなければ
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # 変数の生みの親を格納

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # 勾配を適当に設定

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 取り出し
            x, y = f.input, f.output # 入力と出力を設定
            x.grad = f.backward(y.grad) # バックプロパゲーション

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
# functionメソッドを複数の入力に対応できるようにする
class Function():
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # データを取り出す
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # wrap

        for output in outputs:
            output.set_creator(self) # 生みの親が自分だと覚えさせる
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [5]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # y,でタプルを返す

In [6]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.creator)

# 可変長の引数（改善編）

可変長の引数を使ってシンプルにAddを実装する。

In [12]:
class Function():
    def __call__(self, *inputs): # *をつけて可変長変数にする
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # [x0, x1]を(x0, x1)とアンパッキングして返す
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # wrap

        for output in outputs:
            output.set_creator(self) # 生みの親が自分だと覚えさせる
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [13]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [14]:
def add(x0, x1):
    return Add()(x0, x1)

In [15]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


# ステップ13:可変長の引数を逆伝播でも使う

In [21]:
# addに逆伝播を実装
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

In [23]:
# backwardを複数対応させる
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray): # 形式がndarrayでなければ
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func # 変数の生みの親を格納

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # 勾配を適当に設定

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 取り出し
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,) # タプルに変換
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [24]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [25]:
def square(x):
    f = Square()
    return f(x)

In [26]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [27]:
print(z.grad)

1.0
